<a href="https://colab.research.google.com/github/youhs1125/Kaggle/blob/main/MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import json
from google.colab import drive
drive.mount('/content/drive')

key_path = '/content/drive/MyDrive/Kaggle/Kaggle_key/kaggle.json'

with open(key_path) as f:
  api = json.load(f)
  os.environ["KAGGLE_USERNAME"] = api['username']
  os.environ["KAGGLE_KEY"] = api['key']

Mounted at /content/drive


In [2]:
!kaggle competitions download -c digit-recognizer
!unzip -qq "/content/digit-recognizer.zip"

 85% 13.0M/15.3M [00:01<00:00, 15.1MB/s]
100% 15.3M/15.3M [00:01<00:00, 9.40MB/s]


In [34]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.utils.data as td
import matplotlib.pyplot as plt
import numpy as np
import torchvision.datasets as dsets
import torchvision.transforms as transforms

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [57]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

def train_loop(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  loss_track = np.array([])
  for batch, (X,y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)
    X = X.unsqueeze(dim = 1)
    pred = model(X)
    loss = loss_fn(pred,y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch%100 == 0:
      loss, current = loss.item(),batch*len(X)
      loss_track = np.append(loss_track,np.array(loss))
      print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")
    
  return np.mean(loss_track)

def test_loop(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  test_loss, correct = 0,0

  with torch.no_grad():
    for X, y in dataloader:
      X,y = X.to(device), y.to(device)
      X = X.unsqueeze(dim = 1)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
    test_loss /= num_batches
    print(f"Avg loss: {test_loss:>8f}\n")
  return test_loss

cuda


In [55]:
dataset = dsets.MNIST(root="",train=True,transform=transforms.ToTensor(),download=True)

dataset.data = dataset.data.type(torch.FloatTensor)/255.

dataset = td.TensorDataset(dataset.data,dataset.targets)


trainsize = int(len(dataset)*0.8)
validsize = len(dataset) - trainsize
batchsize = 64

trainset, validset = td.random_split(dataset,[trainsize,validsize])

train_dataloader = td.DataLoader(trainset, batch_size=batchsize, shuffle=True)
valid_dataloader = td.DataLoader(validset, batch_size=batchsize, shuffle=True)

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0., 

In [109]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.linear_relu_stack = nn.Sequential(
        nn.Conv2d(1,64,kernel_size=3,padding=1),
        nn.ReLU(),
        nn.Conv2d(64,64,kernel_size=3,padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,stride=2),
        nn.Conv2d(64,128,kernel_size=3,padding=1),
        nn.ReLU(),
        nn.Conv2d(128,128,kernel_size=3,padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,stride=2),
        nn.Conv2d(128,256,kernel_size=3,padding=1),
        nn.ReLU(),
        nn.Conv2d(256,256,kernel_size=3,padding=1),
        nn.ReLU(),
        nn.Conv2d(256,512,kernel_size=3,padding=1),
        nn.ReLU(),
        nn.Conv2d(512,512,kernel_size=3,padding=1),
        nn.MaxPool2d(kernel_size=2,stride=2),
        nn.ReLU(),
        nn.Flatten(),
        nn.Linear(4608,256),
        nn.ReLU(),
        nn.Linear(256,10),
        nn.Softmax()
    )
  def forward(self, x):
    logits = self.linear_relu_stack(x)
    return logits

model = NeuralNetwork().to(device)

In [111]:
epochs = 80
loss_fn = nn.CrossEntropyLoss()
learning_rate = 1e-5
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
train_loss = np.empty
val_loss = []
for t in range(epochs):
  print(f"Epoch {t+1}\n----------------")
  loss_train = train_loop(train_dataloader, model, loss_fn, optimizer)
  train_loss = np.append(train_loss,loss_train)
  loss_test = test_loop(valid_dataloader, model, loss_fn)
  val_loss.append(loss_test)
print("Done!")

Epoch 1
----------------
loss: 1.960298 [    0/48000]
loss: 1.759228 [ 6400/48000]
loss: 1.774644 [12800/48000]
loss: 1.741794 [19200/48000]
loss: 1.724216 [25600/48000]
loss: 1.719120 [32000/48000]
loss: 1.792878 [38400/48000]
loss: 1.727192 [44800/48000]
Avg loss: 1.724465

Epoch 2
----------------
loss: 1.832512 [    0/48000]
loss: 1.670689 [ 6400/48000]
loss: 1.745089 [12800/48000]
loss: 1.662345 [19200/48000]
loss: 1.700882 [25600/48000]
loss: 1.649306 [32000/48000]
loss: 1.729638 [38400/48000]
loss: 1.628545 [44800/48000]
Avg loss: 1.638880

Epoch 3
----------------
loss: 1.640861 [    0/48000]
loss: 1.693037 [ 6400/48000]
loss: 1.624962 [12800/48000]
loss: 1.680737 [19200/48000]
loss: 1.620701 [25600/48000]
loss: 1.505651 [32000/48000]
loss: 1.555347 [38400/48000]
loss: 1.572353 [44800/48000]
Avg loss: 1.613298

Epoch 4
----------------
loss: 1.607491 [    0/48000]
loss: 1.631871 [ 6400/48000]
loss: 1.633008 [12800/48000]
loss: 1.583043 [19200/48000]
loss: 1.658757 [25600/48000]

In [112]:
test = pd.read_csv("test.csv")
sub = pd.read_csv("sample_submission.csv")
test = torch.FloatTensor(test.to_numpy())

In [113]:
test = test.reshape(-1, 1, 28, 28).to(device)
print(test.shape)

torch.Size([28000, 1, 28, 28])


In [114]:
model.eval()
with torch.no_grad():
  pred = model(test)
  ans = torch.argmax(pred,dim=1)

print(ans)

tensor([2, 0, 9,  ..., 3, 9, 2], device='cuda:0')


In [115]:
sub['Label'] = ans.cpu()
print(sub)

       ImageId  Label
0            1      2
1            2      0
2            3      9
3            4      0
4            5      3
...        ...    ...
27995    27996      9
27996    27997      7
27997    27998      3
27998    27999      9
27999    28000      2

[28000 rows x 2 columns]


In [116]:
sub.to_csv("predict.csv",index=False)

In [117]:
!kaggle competitions submit -c digit-recognizer -f predict.csv -m "AUTO_SUBMIT"

100% 208k/208k [00:03<00:00, 69.2kB/s]
Successfully submitted to Digit Recognizer